## What is the most optimal skill to learn for Data Analysts?

### Methodology
1. Continue from last notebook to find percent of postings with skill
2. Visualize median salary vs percent skill demand
3. (optional) Determine if certain technologies are more prevalent